# Kennismatcher: Match RIO beroepen aan ESCO beroepen

## Doel: Festivalgangers o.a. kunnen koppelen op basis van hun functie en competenties

### In de isco/esco codering van beroepen zijn belangrijke competenties gekoppeld aan 2940 beroepen. Door deze beroepen te koppelen aan de functies binnen de gemeente, kunnen de competenties die personen bezitten achterhaald worden. Bij een volgende kennisfestival, kunnen mensen gekoppeld worden op basis van hun kennisvraag aan mensen die de competenties bezitten om de vraag te kunnen beantwoorden.

#### Code geschreven door: Rukshar Wagid Hosain, rfg.wagidhosain@rotterdam.nl

## Download necessary packages

In [ ]:
!pip install pandas
!pip install spacy
!pip install xlrd
!pip install pyarrow
!python -m spacy download nl_core_news_lg

## Import libraries

In [2]:
import pandas as pd
import spacy

## Import datasets

In [6]:
df_rio_functietitels = pd.read_csv('Data/20200227_RIO_functietitels.csv', encoding = "ISO-8859-1", sep=';')
df_steekwoorden= pd.read_excel('Data/kennismatcher_expert_leren.xlsx')
df_steekwoorden = df_steekwoorden.dropna()
#df_esco = pd.read_feather('Data/joined_hierarchies_occupations_skills.feather')
df_esco = pd.read_feather('Data/joined_occupations_skills_sample.feather')
df_esco = df_esco[df_esco['preferredLabel_occupations'].notna()]

### Explore datasets

In [7]:
df_rio_functietitels.head()

,functie
0,Teamleider beveiliging
1,Medewerker Gegevensbeheer
2,Assistent bewindvoerder
3,Chauffeur inzameling A
4,Teamleider


In [8]:
df_steekwoorden.head()

,Onderwerp - expert,Onderwerp - leren
2,Management|onderwijsonderzoek|bachelor onderwijs|,big data|rotterdamcirculair|toekomst|
3,gezondheid|jongeren|onderzoek|,gezondheid|jongeren|onderzoek|
4,strategie|audit|ITGC|,coaching|samenwerken|trainingen|
7,ondernemen|vergunningen|bedrijfshuisvesting|,21e-eeuwse vaardigheden|sociaal ondernemerscha...
8,kennisinfrastructuur|lvb|traineeship|,deep democracy|omgevingswet|verandermanagement|


In [9]:
df_esco.head()

,preferredLabel_occupations,preferredLabel_level4,preferredLabel_skills
0,admiraal,Officieren,marineoperaties
1,admiraal,Officieren,militaire code
2,admiraal,Officieren,scheepsvloot beheren
3,admiraal,Officieren,zorgen voor de openbare orde en veiligheid
4,admiraal,Officieren,mensenrechten verdedigen


## Load and prepare language model

In [10]:
nlp = nl_core_news_lg.load()
# nlp = spacy.load(nl_core_news_lg)

In [11]:
# Count the number of RIO functions that don't have a vector
count = 0
for word in df_rio_functietitels['functie'].unique():
    doc = nlp(word)
    if doc.vector_norm == 0:
        count+=1
print(len(df_rio_functietitels['functie'].unique()))
print(count)

894
160


In [12]:
# Count the number of ESCO functions that don't have a vector
count = 0
for occupation in df_esco['preferredLabel_occupations'].unique():
    doc = nlp(occupation)
    if doc.vector_norm == 0:
        count+=1
print(len(df_esco['preferredLabel_occupations'].unique()))
print(count)

2941
639


## Create or load similarity matrix

In [14]:
# UNCOMMENT IF YOU WANT TO CREATE THE SIMILARITY MATRIX YOURSELF

# similarity_matrix = np.zeros((len(df_esco['preferredLabel_occupations'].unique()), 
#                               len(df_rio_functietitels['functie'].unique())))

# for i, occupation in enumerate(df_esco['preferredLabel_occupations'].unique()):
#     doc1 = nlp(occupation)
#     for j, rio_occupation in enumerate(df_rio_functietitels['functie'].unique()):
#         doc2 = nlp(rio_occupation)
#         similarity_score = doc1.similarity(doc2)
#         similarity_matrix[i][j] = similarity_score
#     print(i)

# print(similarity_matrix)

# # Numpy similarity matrix to dataframe
# similarity_df = pd.DataFrame(similarity_matrix, columns = df_rio_functietitels['functie'].unique())
# similarity_df['beroep'] = df_esco['preferredLabel_occupations'].unique()

similarity_df = pd.read_excel('Data/similarity_matrix_beroepen.xlsx')
similarity_df

,Teamleider beveiliging,Medewerker Gegevensbeheer,Assistent bewindvoerder,Chauffeur inzameling A,Teamleider,Medewerker klantcontact,Medewerker beveiliging,Administratieve ondersteuning,Projectmanager,Bouwinspecteur,...,Beheerder distributiecentrum,Bureauhoofd Rekenkamer,Directeur FBZN,Teamleider opslag,Medewerker EPC B,Coördinator VPI & VPO,Procesregisseur/plv manager,Adviseur C,Analist,beroep
0,0.199359,0.092040,0.277149,0.052464,0.217043,0.074129,0.124364,0.098419,0.074641,0,...,0.193135,0.253664,0.248017,0.151712,-0.010153,-0.101770,0.185122,0.018745,0.208241,admiraal
1,0.499797,0.405392,0.605462,0.169861,0.385228,0.359404,0.477274,0.401574,0.210854,0,...,0.490221,0.403460,0.392460,0.418789,0.146326,-0.072126,0.312392,0.135341,0.295829,officier zeedienst
2,0.333943,0.180045,0.427554,0.104438,0.259174,0.155703,0.279557,0.202476,0.093834,0,...,0.321428,0.322113,0.288571,0.286109,0.077458,-0.122032,0.200815,0.073404,0.219122,officier artillerie
3,0.188626,-0.002802,0.273587,0.023334,0.172560,0.005391,0.110428,0.015541,-0.000183,0,...,0.195939,0.205174,0.152888,0.176513,-0.007951,-0.221804,0.124322,-0.007381,0.228939,legerkapitein
4,0.377835,0.266838,0.440223,0.140351,0.375941,0.217576,0.297935,0.262787,0.227592,0,...,0.324711,0.246929,0.382249,0.302676,0.080062,-0.007259,0.293542,0.120484,0.314377,luitenant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2936,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,attractiemedewerker
2937,0.326178,0.278839,0.353861,0.020590,0.159599,0.339731,0.378232,0.297035,0.153064,0,...,0.407250,0.184852,0.123797,0.316343,0.010669,-0.092020,0.262039,-0.020563,0.207010,"servicemedewerker bioscoop, theater"
2938,0.322384,0.267257,0.344113,0.118751,0.123206,0.232975,0.339483,0.419504,0.110131,0,...,0.418792,0.211351,0.121413,0.332324,0.169694,-0.082666,0.056126,0.047894,0.149869,plaatser van affiches
2939,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,wasserettemedewerker


In [15]:
# Find the most similar occupations and use as benchmark
most_similar_esco_function = []
for max_location in similarity_df.drop('beroep', axis=1).idxmax(axis = 0):
    most_similar_esco_function.append(similarity_df.loc[max_location]['beroep'])

df_most_similar_benchmark = pd.DataFrame({'rio_functie': similarity_df.drop('beroep', axis=1).idxmax(axis = 0).keys(), 
             'esco_beroep': most_similar_esco_function} )

In [16]:
df_most_similar_benchmark

,rio_functie,esco_beroep
0,Teamleider beveiliging,coördinator beveiliging
1,Medewerker Gegevensbeheer,administratief medewerker ledenadministratie
2,Assistent bewindvoerder,administratief medewerker
3,Chauffeur inzameling A,coördinator internationaal transport
4,Teamleider,teamleider kabelleggers
...,...,...
889,Medewerker EPC B,energiedeskundige
890,Coördinator VPI & VPO,coördinator milieuprogramma's
891,Procesregisseur/plv manager,manager dierenfaciliteit
892,Adviseur C,procesoperator zeeppoederproductie


## Export new datasets

In [179]:
similarity_df.to_excel('similarity_matrix_beroepen.xlsx')
df_most_similar_benchmark.to_excel('most_similar_beroepen_benchmark.xlsx')